# Analyse van AskDelphi Relations & Tags endpoint
Dit notebook ondersteunt het verkennen, analyseren en valideren van de Relations & Tags functionaliteit binnen AskDelphi.  
De module biedt helpermethoden voor het toevoegen, beheren en inspecteren van relaties en tags die aan AskDelphi‑topics gekoppeld zijn.  
Het doel van dit notebook is om:

- inzicht te krijgen in de datastructuren en API‑interacties rondom relations & tags
- voorbeelddata te analyseren en patronen te ontdekken
- helpermethoden te testen en gedrag te evalueren
- experimentele of onderzoeksgerichte analyses uit te voeren ter verbetering van de contentstructuur in AskDelphi

Dit notebook vormt daarmee een flexibel startpunt voor verdere exploratie of debugging van de Relations & Tags endpoint.

### Initialisatie omgeving

In [1]:
import sys
import uuid
from pathlib import Path

project_root = Path.cwd().parent          # ../ vanaf Notebooks/
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [2]:
from pathlib import Path

# Vind de projectroot vanuit Notebooks/
project_root = Path.cwd().parent
src_path = project_root / "src"

# Voeg src/ toe aan modulezoekpad
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print("Project root:", project_root)
print("SRC path:", src_path)
print("SRC path is in sys.path?", str(src_path) in sys.path)

Project root: /Users/baasa03/projects/ask-delphi-api
SRC path: /Users/baasa03/projects/ask-delphi-api/src
SRC path is in sys.path? True


### Connectie met AskDelphi opzetten

In [3]:
from askdelphi.authentication import AskDelphiClient
client = AskDelphiClient()
client.authenticate()   # pakt automatisch portal code uit .env

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/Users/baasa03/projects/ask-delphi-api/.venv/lib/python3.9/site-packages/traitlets/traitlets.py", line 632, in get
    value = obj._trait_values[self.name]
KeyError: '_control_lock'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/baasa03/projects/ask-delphi-api/.venv/lib/python3.9/site-packages/zmq/eventloop/zmqstream.py", line 565, in _log_error
    f.result()
  File "/Users/baasa03/projects/ask-delphi-api/.venv/lib/python3.9/site-packages/ipykernel/kernelbase.py", line 301, in dispatch_control
    async with self._control_lock:
  File "/Users/baasa03/projects/ask-delphi-api/.venv/lib/python3.9/site-packages/traitlets/traitlets.py", line 687, in __get__
    return t.cast(G, self.get(obj, cls))  # the G should encode the Optional
  File "/Users/baasa03/projects/ask-delphi-api/.venv/lib/python3.9/site-packages/t

AskDelphi Client loaded.
Parsed tenant/project/acl from CMS URL
Loaded cached tokens.
AUTHENTICATION STARTED
Trying cached tokens...
Editing API token status: 200
Editing API token acquired.
SUCCESS using cached tokens!


True

In [4]:
from askdelphi.project import Project
from askdelphi.topictools import TopicTools
from askdelphi.relation import Relation
project = Project(client)
topic = TopicTools(client, project)
relation = Relation(client)

### Creatie Digicoach

In [ ]:
# Constant
DIGICOACH_NAME = "Digicoach test arie 5feb"
TASK_NAME = "Taak test 5feb"

# Create voorgedefinieerde zoekopdracht

# Create Digicoach topic
topic_id_digicoach = topic.topic_upload(DIGICOACH_NAME, "Digitale Coach Procespagina")

# Get Digicoach topic_version_id, task_relation_id
topic_version_id = topic.get_topicVersionId(topic_id_digicoach)
print(f"Created Digicoach topic : {topic_id_digicoach}")
print(f"Created Digicoach topic version id: {topic_version_id}")

# Add Tag Keten aan proces

# Get parent relatietype ID
parentTopicRelationTypeId = relation.get_task_relation_id(topic_id_digicoach, topic_version_id)

# Create Task topic
topicId_task = str(uuid.uuid4())                    # ID van het doel-topic.        
topicTitle = TASK_NAME                              # Titel van het doel-topic.        
topicTypeId = project.get_topic_type_id("Task")     # ID van het topictype.        
parentTopicId = topic_id_digicoach                  # ID van het bron-topic.
parentTopicRelationTypeId = parentTopicRelationTypeId   # ID van het relatietype
parentTopicVersionId = topic_version_id             # Versie-ID van het bron-topic

relation.add_topic_with_relation(
    client, 
    topicId_task, 
    topicTitle, 
    topicTypeId, 
    parentTopicId,
    parentTopicRelationTypeId, 
    parentTopicVersionId)

print(f"Created Task topic : {topicId_task}")

# Delete Task topic
topic_version_id_task = topic.get_topicVersionId(topicId_task)
topic.delete_topic(topicId_task, topic_version_id_task)
print(f"Deleted Task topic : {topicId_task}")

# Delete Digicoach topic
topic.delete_topic(topic_id_digicoach, topic_version_id)
print(f"Deleted Digicoach topic : {topic_id_digicoach}")


REQUEST: GET https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/contentdesign
RESPONSE: 200 OK

REQUEST: POST https://edit.api.askdelphi.com/v4/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic
JSON body: {
  "topicTitle": "Digicoach test arie 5feb",
  "topicTypeId": "7d332fbb-44f5-469f-b570-874e701e526b"
}
RESPONSE: 200 OK

REQUEST: POST https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic/29e17f2f-cf01-46d3-a24c-c487f7399182/workflowstate
JSON body: {
  "action": 1
}
RESPONSE: 200 OK
Created Digicoach topic : 29e17f2f-cf01-46d3-a24c-c487f7399182
Created Digicoach topic version id: 336e7afc-1c0f-42c1-b97d-4686e0c4fd0f

REQUEST: GET https://edit.api.askdelphi.com/v2/tenant/0